This code takes input images from an opencv function called *opencv_grabber*; it sends through still images from a webcam at 25+ frames per second. The program outputs these images to a yarpview port name */internalView*. It also takes commands from a RPC port; if this port sends the command **capture**, the program will output the latest image as a picture to another yarpview port, named */pictureView*. 

The program exports the yarp images received into a numpy array (*imgArray*). This allows for the data received to be manipulated. To prove that this can be done, the program adds a 50x50pixel block of black on the top left corner of the output video and image. 

The following commands need to be executed in separate terminal windows prior to running this program:

* yarp server
* yarpview --name /internalView
* yarpview --name /pictureView
* yarpdev --device opencv_grabber

The following command needs to be run in yet another terminal window after this program is run

* yarp rpc /imageRPC

NB: This is for me (the author): Check if the refresh rate of the updateModule() method needs to match the frame rate of the data send from opencv_grabber. When testing this code, it wasn't necessary, but maybe that was a fluke, and I just got lucky. Check!

In [1]:
import readline
import yarp
import sys
import time
import         
        #
        # for i in range(0,h-1):
        #     for j in range(0,w-1):
        #         self.imgBuffer[i,j] = self.imgArray[h-1,w-1-j]
        # self.imgArray = self.imgBuffernumpy
import cv2

In [2]:
numpy.set_printoptions(threshold='nan')

class pythonRFMod(yarp.RFModule):

    def __init__(self):
        yarp.RFModule.__init__(self)
        self.respondPort = None
        self.inputPort = None
        self.outputStillPort = None
        self.outputImagePort = None
        self.inputImage = None
        self.outputImage = None
        self.outputStill = None
        self.uniCom = None
        self.imgArray = None
        self.tempImage = None

    def configure(self, rf):
        yarp.Network.init()

        self.respondPort = yarp.Port()
        self.inputPort = yarp.BufferedPortImageRgb()
        self.outputStillPort = yarp.BufferedPortImageRgb()
        self.outputImagePort = yarp.BufferedPortImageRgb()

        self.respondPort.open('/imageRPC')
        self.inputPort.open('/pythonRead')
        self.outputImagePort.open('/imageWrite')
        self.outputStill        
        #
        # for i in range(0,h-1):
        #     for j in range(0,w-1):
        #         self.imgBuffer[i,j] = self.imgArray[h-1,w-1-j]
        # self.imgArray = self.imgBufferPort.open('/pictureWrite')

        self.attach(self.respondPort)

        yarp.Network.connect('/grabber', '/pythonRead')
        yarp.Network.connect('/pictureWrite', '/pictureView')
        yarp.Network.connect('/imageWrite', '/internalView')

        self.imgArray = numpy.zeros((240, 320, 3), dtype=numpy.uint8)
        self.imgBuffer = numpy.zeros((240, 320, 3), dtype=numpy.uint8)

        self.inputImage = yarp.ImageRgb()

        self.tempImage = yarp.ImageRgb()
        self.tempImage.resize(320, 240)
        self.tempImage.setExternal(self.imgArray, self.imgArray.shape[1], self.imgArray.shape[0])

        self.outputImage = yarp.ImageRgb()
        self.outputStill = yarp.ImageRgb()

        self.uniCom = 'None'

        print 'configured'
        return True

    def respond(self, command, reply):
        self.uniCom = command.toString()
        reply.clear()
        reply.addString('Recieved command')

        if self.uniCom == 'capture':

            self.outputStill = self.outputStillPort.prepare()
            self.outputStill.copy(self.tempImage)
            self.outputStillPort.write()

            self.uniCom = 'None'

        return True

    def getPeriod(self):
        
        #Need to check if this needs to match with the frame rate of the input images.
        return 0.03

    def updateModule(self):
        self.inputImage = self.inputPort.read()
        h = self.inputImage.height()
        w = self.inputImage.width()

        if (h != self.tempImage.height() or w != self.tempImage.width()):
            self.imgArray        
        #
        # for i in range(0,h-1):
        #     for j in range(0,w-1):
        #         self.imgBuffer[i,j] = self.imgArray[h-1,w-1-j]
        # self.imgArray = self.imgBuffer = numpy.zeros((h, w, 3), dtype=numpy.uint8)
            self.imgBuffer = numpy.zeros((h, w, 3), dtype=numpy.uint8)
            self.tempImage.resize(w, h)
            self.tempImage.setExternal(self.imgArray, self.imgArray.shape[1], self.imgArray.shape[0])

        self.tempImage.copy(self.inputImage)

        self.imgArray[:50, :50] = 0

        self.outputImage = self.outputImagePort.prepare()
        self.outputImage.copy(self.tempImage)
        self.outputImagePort.write()
        return True

    def interruptModule(self):
        return True

    def close(self):
        self.inputPort.close()
        self.outputPort.close()
        return True

In [ ]:
if __name__ == '__main__':
    yarp.Network.init()
    mod = pythonRFMod()
    yrf = yarp.ResourceFinder()
    yrf.setVerbose(True)

    yrf.configure(sys.argv)
    mod.runModule(yrf)